In [1]:
#Load Library 
import numpy as np
import pandas as pd
import os
import argparse
import errno
import scipy.misc
import dlib
import cv2

from skimage.feature import hog

In [2]:
#Parameter 
image_height = 48
image_width = 48
window_size = 24
window_step = 6
SELECTED_LABELS = []
IMAGES_PER_LABEL = 500
OUTPUT_FOLDER = os.getcwd() + "/Dataset/Image_Features"
predictor_path =os.getcwd() + "/Dlib/shape_predictor_68_face_landmarks.dat"
ONE_HOT_ENCODING = True

In [3]:
# loading Dlib predictor and preparing arrays:
print( "Preparing")
SELECTED_LABELS = [0,1,2,3,4,5,6]
predictor = dlib.shape_predictor() # Dlib 68 Face Landmark
original_labels = [0, 1, 2, 3, 4, 5, 6]
new_labels = list(set(original_labels) & set(SELECTED_LABELS))
nb_images_per_label = list(np.zeros(len(new_labels), 'uint8'))

Preparing


In [4]:
#For getting 68 face landmarks from the image
def get_landmarks(image, rects):
    # this function have been copied from http://bit.ly/2cj7Fpq
    if len(rects) > 1:
        raise BaseException("TooManyFaces")
    if len(rects) == 0:
        raise BaseException("NoFaces")
    return np.matrix([[p.x, p.y] for p in predictor(image, rects[0]).parts()])

In [5]:
#For getting Hog 
def sliding_hog_windows(image):
    hog_windows = []
    for y in range(0, image_height, window_step):
        for x in range(0, image_width, window_step):
            window = image[y:y+window_size, x:x+window_size]
            hog_windows.extend(hog(window, orientations=8, pixels_per_cell=(8, 8),
                                            cells_per_block=(1, 1), visualise=False))
    return hog_windows

In [7]:
data = pd.read_csv(os.getcwd() + "/Dataset/fer2013.csv")

In [9]:
for category in data['Usage'].unique():
    print( "converting set: " + category + "...")
    # create folder
    os.makedirs(OUTPUT_FOLDER + '/' + category)
    # get samples and labels of the actual category
    category_data = data[data['Usage'] == category]
    samples = category_data['pixels'].values
    labels = category_data['emotion'].values
    
    # get images and extract features
    images = []
    labels_list = []
    landmarks = []
    hog_features = []
    hog_images = []
    
    for i in range(len(samples)):
        image = np.fromstring(samples[i], dtype=int, sep=" ").reshape((image_height, image_width))
        images.append(image)
        #scipy.misc.imsave(category + '/' + str(i) + '.jpg', image)
        features = sliding_hog_windows(image)
        
        f, hog_image = hog(image, orientations=8, pixels_per_cell=(16, 16),
        cells_per_block=(1, 1), visualise=True)
        hog_features.append(features)
        
        scipy.misc.imsave('temp.jpg', image)
        image2 = cv2.imread('temp.jpg')
        face_rects = [dlib.rectangle(left=1, top=1, right=47, bottom=47)]
        face_landmarks = get_landmarks(image2, face_rects)
        landmarks.append(face_landmarks)            
        labels_list.append(labels[i])
    
    np.save(OUTPUT_FOLDER + '/' + category + '/labels.npy', labels_list)
    np.save(OUTPUT_FOLDER + '/' + category + '/landmarks.npy', landmarks)
    np.save(OUTPUT_FOLDER + '/' + category + '/hog_features.npy', hog_features)
    np.save(OUTPUT_FOLDER+ '/' + category + '/images.npy', images)

converting set: Training...


/anaconda3/lib/python3.6/site-packages/skimage/feature/_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:27: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.


converting set: PublicTest...
converting set: PrivateTest...
